<a href="https://colab.research.google.com/github/startbkgogo/first/blob/main/ch03_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 파일 다운 받기
import gdown
gdown.download("https://bit.ly/3RhoNho", "ns_202104.csv", quiet=False)


Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:00<00:00, 211MB/s]


'ns_202104.csv'

In [4]:
# pandas로 데이터 불러오기
import pandas as pd
ns_df = pd.read_csv('ns_202104.csv', low_memory=False)
ns_df.head()
# 의미없는 열 제거
ns_book = ns_df.loc[:, "번호":"등록일자"]

In [7]:
selected_rows = ns_df["출판사"] == "한빛미디어"
ns_book2 = ns_book[selected_rows]
count_df = ns_book[['도서명','저자','ISBN', '권', '대출건수']]
group_df = count_df.groupby(by=['도서명','저자','ISBN', '권'], dropna=False)
loan_count = group_df.sum()
# 도서 대여가 중복이 되는 데이터
# 처음 데이터는 중복이 되지 않았다 - False, 2번 데이터부터는 중복이 되었다. - True
dup_rows = ns_book.duplicated(subset=["도서명","저자","ISBN","권"]) # 중복된 행을 True 로 표시한다.
# 중복이 안되는 데이터
unique_rows = ~dup_rows # 불리언 배열을 반전시켜 고유한 행을 True 로 표시한다.
ns_book3 = ns_book[unique_rows].copy() # 고유한 행만 선택한다.
ns_book3.set_index(["도서명","저자","ISBN","권"], inplace=True)
ns_book3.update(loan_count)
ns_book4 = ns_book3.reset_index()
ns_book4.head()

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈 지음,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형 지음,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,9788934990833,NaN,5,김영사,2021,NaN,NaN,NaN,1,0,2021-03-19


In [13]:
# 정규 표현식을 이용한 데이터 수정
# replace({"키:항목":{키 찾는데이터:바꿀데이터}})
# 찾는 데이터를 정규 표현식 활용한 바꾸기
# 일반적인 replace
# ns_book4.replace({"발행년도":{"2021":"21"}})[100:102]
# 정규표현식을 이용한 replace : 한번에 yyyy -> yy 형식으로 바꾼다. r"문자열" : 문자열이 정규 표현식
# 정규 표현식의 \d -> 숫자 데이터() - 그룹으로 만듬.
# r"\1" - 첫번째 그룹의 데이터
ns_book4.replace({"발행년도":{r"\d\d(\d\d)":r"\1"}}, regex=True)[100:102]


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",9788958743019,NaN,101,좋은씨앗,18,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",9788960518483,NaN,102,부키,21,NaN,NaN,325.1,1,0,2021-03-15


In [14]:
# 정규 표현식이 반복될 때는 중괄호를 사용하여 개수를 지정한다.
ns_book4.replace({"발행년도":{r"\d{2}(\d{2})":r"\1"}}, regex=True)[100:102]

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드 (지은이), 김진웅 (옮긴이)",9788958743019,NaN,101,좋은씨앗,18,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아 (지은이), 안기순 (옮긴이)",9788960518483,NaN,102,부키,21,NaN,NaN,325.1,1,0,2021-03-15


In [15]:
# 저장 항목에서 (지은이), (옮긴이) 제거해 보자.
# () - 그룹으로 처리 \(\) - 순수 문자열 ()로 처리
# . - 모든 문자, \s : 공백문자
# (.*) - 지은이 앞에 사람이름 : 첫번째 그룹 - \1, (.*) - 옮긴이 앞에 사람이름 + 앞에 "." + "blk" : 두번째 그룹 - \2
ns_book4.replace({"저자": {r"(.*)\s\(지은이\)(.*)\s\(옮긴이\)": r"\1\2"},
                  "발행년도": {r"\d{2}(\d{2})": r"\1"}}, regex=True)[100:102]

,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
100,No라고 말할 줄 아는 남편과 아내 - 개정판,"헨리 클라우드, 존 타운센드, 김진웅",9788958743019,NaN,101,좋은씨앗,18,NaN,NaN,234.9,1,1,2021-03-15
101,"D2C 레볼루션 - 스타트업부터 글로벌 기업까지, 마켓 체인저의 필수 전략","로런스 인그래시아, 안기순",9788960518483,NaN,102,부키,21,NaN,NaN,325.1,1,0,2021-03-15


In [24]:
ns_book4.replace({"저자":{r"(.*)\s지음(.*)\s옮김": r"\1\2"}}, regex=True).replace({"저자":{r"(.*)\s지음": r"\1"}}, regex=True)[:4]


,도서명,저자,ISBN,권,번호,출판사,발행년도,세트 ISBN,부가기호,주제분류번호,도서권수,대출건수,등록일자
0,인공지능과 흙,김동훈,9788937444319,NaN,1,민음사,2021,NaN,NaN,NaN,1,0,2021-03-19
1,가짜 행복 권하는 사회,김태형,9791190123969,NaN,2,갈매나무,2021,NaN,NaN,NaN,1,0,2021-03-19
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영,9788968332982,NaN,3,블랙피쉬,2021,NaN,NaN,NaN,1,0,2021-03-19
3,예루살렘 해변,"이도 게펜, 임재희",9788970759906,NaN,4,문학세계사,2021,NaN,NaN,NaN,1,0,2021-03-19
